In [1]:
import pandas as pd
import json
from pandas import json_normalize
from tqdm import tqdm_notebook as tqdm
import numpy as np
import ast


pd.set_option('display.precision',6)
pd.set_option('display.max_columns', None)
pd.options.mode.chained_assignment = None

In [2]:
fighters_df = pd.read_csv('data/0.fighters_raw.csv', index_col=0)
fighters_df['dateOfBirth'] = pd.to_datetime(fighters_df['dateOfBirth'])
fighters_df['birthYear'] = fighters_df['dateOfBirth'].dt.year
use_cols = ['name', 'armSpan', 'city', 'country', 'dateOfBirth',
       'height', 'legSwing', 'weight', 'timezone', 'weightCategory.name']

fighters_df[use_cols].isna().sum()

name                      0
armSpan                1942
city                   2308
country                1211
dateOfBirth              70
height                  334
legSwing               2895
weight                  139
timezone               1361
weightCategory.name       0
dtype: int64

In [3]:
df = pd.read_csv('data/0.events_raw.csv', index_col=0)
df = df.set_index('id').sort_index()

df['eventDate.date'] = pd.to_datetime(df['eventDate.date'])
df = df.sort_values('eventDate.date').reset_index()
df['fighters'] = df['fighters'].apply(lambda x: ast.literal_eval(x))

df

id                                            avgOdds       city  \
0      5208                                                 []     Denver   
1      5207                                                 []     Denver   
2      5206                                                 []     Denver   
3      5205                                                 []     Denver   
4      5204                                                 []     Denver   
...     ...                                                ...        ...   
7095  40493  [{'fighterId': 348, 'value': 1.95}, {'fighterI...  Las Vegas   
7096  40492  [{'fighterId': 53, 'value': 2}, {'fighterId': ...  Las Vegas   
7097  40507  [{'fighterId': 145, 'value': 3.95}, {'fighterI...  Las Vegas   
7098  40453  [{'fighterId': 628, 'value': 2.1}, {'fighterId...  Las Vegas   
7099  40452  [{'fighterId': 246, 'value': 2.75}, {'fighterI...  Las Vegas   

      completed country  duration      eventDate.date eventDate.timezone  \
0          True     USA      26.0 1993-11-12 00:00:00      Europe/Berlin   
1          True     USA     260.0 1993-11-12 00:00:00      Europe/Berlin   
2          True     USA     109.0 1993-11-12 00:00:00      Europe/Berlin   
3          True     USA     138.0 1993-11-12 00:00:00      Europe/Berlin   
4          True     USA      57.0 1993-11-12 00:00:00      Europe/Berlin   
...         ...     ...       ...                 ...                ...   
7095       True     USA     300.0 2021-02-27 00:00:00      Europe/Berlin   
7096      False     USA       NaN 2021-02-27 01:45:00      Europe/Berlin   
7097      False     USA       NaN 2021-02-27 03:45:00      Europe/Berlin   
7098       True     USA     295.0 2021-03-06 00:00:00      Europe/Berlin   
7099       True     USA     208.0 2021-03-06 00:00:00      Europe/Berlin   

      eventDate.timezone_type  fighterId_1  fighterId_2  \
0                           3         1923         1925   
1                           3         1870         1908   
2                           3         1631         1895   
3                           3         1646         1924   
4                           3         1631         1646   
...                       ...          ...          ...   
7095                        3          348          651   
7096                        3           53          275   
7097                        3          145          499   
7098                        3          628          687   
7099                        3          246         2073   

                                               fighters  \
0     [{'fighterId': 1923, 'fightStats': {'hitsTotal...   
1     [{'fighterId': 1870, 'fightStats': {'hitsTotal...   
2     [{'fighterId': 1631, 'fightStats': {'hitsTotal...   
3     [{'fighterId': 1646, 'fightStats': {'hitsTotal...   
4     [{'fighterId': 1631, 'fightStats': {'hitsTotal...   
...                                                 ...   
7095  [{'fighterId': 348, 'fightStats': {'hitsTotal'...   
7096  [{'fighterId': 53, 'fightStats': [], 'roundSta...   
7097  [{'fighterId': 145, 'fightStats': [], 'roundSt...   
7098  [{'fighterId': 628, 'fightStats': {'hitsTotal'...   
7099  [{'fighterId': 246, 'fightStats': {'hitsTotal'...   

                                                   link             name  \
0     http://www.ufcstats.com/fight-details/567a09fd...            UFC 1   
1     http://www.ufcstats.com/fight-details/2d2bbc86...            UFC 1   
2     http://www.ufcstats.com/fight-details/46acd54c...            UFC 1   
3     http://www.ufcstats.com/fight-details/cecdc0da...            UFC 1   
4     http://www.ufcstats.com/fight-details/ffd16691...            UFC 1   
...                                                 ...              ...   
7095  http://www.ufcstats.com/fight-details/f3ad8ad8...  UFC Fight Night   
7096  http://www.ufcstats.com/fight-details/dd3dc5fe...  UFC Fight Night   
7097  http://www.ufcstats.com/fight-details/8ad9b2f0...  UFC Fight Ni

In [4]:
def parse_odds(df):  # Parse odds from avgOdds dict
    # Добавить кэффы в df
    for i in df.index[:]:
        avgodds = df.loc[i, 'avgOdds']

        if avgodds == '[]':
            continue

        # Преобразую данные о коэффах из строки
        ids1, odd1 = int(avgodds.split()[1].split(',')[0]), float(avgodds.split()[3].split('}')[0])
        ids2, odd2 = int(avgodds.split()[5].split(',')[0]), float(avgodds.split()[7].split('}')[0])

        # Проверить совпадает ли fighterId_1 с ids1 (который в avgOdds)
        if df.loc[i, 'fighterId_1'] == ids1:
            df.at[i, 'odd1'] = odd1
            df.at[i, 'odd2'] = odd2

        if df.loc[i, 'fighterId_1'] == ids2:
            df.at[i, 'odd1'] = odd2
            df.at[i, 'odd2'] = odd1
    
    return df.drop('avgOdds', axis=1)

df = parse_odds(df)
df = df[~df['odd1'].isna()].reset_index()

# When fighterID_1 wins, then Winner == 1 
df['winner'] = df['winnerId'] == df['fighterId_1']

df

index     id        city  completed country  duration  \
0      1564   4273   Las Vegas       True     USA      61.0   
1      2481   3667  Louisville       True     USA     300.0   
2      3262   2969      Newark       True     USA     300.0   
3      3263   2970      Newark       True     USA     273.0   
4      3264   2968      Newark       True     USA     300.0   
...     ...    ...         ...        ...     ...       ...   
3729   7095  40493   Las Vegas       True     USA     300.0   
3730   7096  40492   Las Vegas      False     USA       NaN   
3731   7097  40507   Las Vegas      False     USA       NaN   
3732   7098  40453   Las Vegas       True     USA     295.0   
3733   7099  40452   Las Vegas       True     USA     208.0   

          eventDate.date eventDate.timezone  eventDate.timezone_type  \
0    2008-07-19 00:00:00      Europe/Berlin                        3   
1    2011-03-03 00:00:00      Europe/Berlin                        3   
2    2013-04-27 00:00:00      Europe/Berlin                        3   
3    2013-04-27 00:00:00      Europe/Berlin                        3   
4    2013-04-27 00:00:00      Europe/Berlin                        3   
...                  ...                ...                      ...   
3729 2021-02-27 00:00:00      Europe/Berlin                        3   
3730 2021-02-27 01:45:00      Europe/Berlin                        3   
3731 2021-02-27 03:45:00      Europe/Berlin                        3   
3732 2021-03-06 00:00:00      Europe/Berlin                        3   
3733 2021-03-06 00:00:00      Europe/Berlin                        3   

      fighterId_1  fighterId_2  \
0             503         1429   
1             983         1366   
2            1181         1231   
3             122         1188   
4             428         1136   
...           ...          ...   
3729          348          651   
3730           53          275   
3731          145          499   
3732          628          687   
3733          246         2073   

                                               fighters  \
0     [{'fighterId': 503, 'fightStats': {'hitsTotal'...   
1     [{'fighterId': 983, 'fightStats': {'hitsTotal'...   
2     [{'fighterId': 1181, 'fightStats': {'hitsTotal...   
3     [{'fighterId': 122, 'fightStats': {'hitsTotal'...   
4     [{'fighterId': 428, 'fightStats': {'hitsTotal'...   
...                                                 ...   
3729  [{'fighterId': 348, 'fightStats': {'hitsTotal'...   
3730  [{'fighterId': 53, 'fightStats': [], 'roundSta...   
3731  [{'fighterId': 145, 'fightStats': [], 'roundSt...   
3732  [{'fighterId': 628, 'fightStats': {'hitsTotal'...   
3733  [{'fighterId': 246, 'fightStats': {'hitsTotal'...   

                                                   link             name  \
0     http://www.ufcstats.com/fight-details/4eafb265...              UFC   
1     http://www.ufcstats.com/fight-details/bbbb4c62...    UFC on VERSUS   
2     http://www.ufcstats.com/fight-details/44ad16cd...          UFC 159   
3     http://www.ufcstats.com/fight-details/8151421e...          UFC 159   
4     http://www.ufcstats.com/fight-details/6d6a1834...          UFC 159   
...                                                 ...              ...   
3729  http://www.ufcstats.com/fight-details/f3ad8ad8...  UFC Fight Night   
3730  http://www.ufcstats.com/fight-details/dd3dc5fe...  UFC Fight Night   
3731  http://www.ufcstats.com/fight-details/8ad9b2f0...  UFC Fight Night   
3732  http://www.ufcstats.com/fight-details/873306ad...          UFC 259   
3733  http://www.ufcstats.com/fight-details/7ad09f88...          UFC 259   

      rounds             timezone  weightCategory.id  \
0        1.0  America/Los_Angeles                  8   
1        3.0      America/Chicago                  5   
2        3.0        Europe/London                  4   
3        1.0        Europe/London                  8   
4        3.0        Europe/London                  4   
...      ...     

In [5]:
def parse_data_from_fight(fightStats, duration):
    if len(fightStats) == 0:
        return [np.nan]*23
    
    hitsTotal = fightStats.get('hitsTotal')
    hitsSuccessful = fightStats.get('hitsSuccessful')
    accentedHitsTotal = fightStats.get('accentedHitsTotal')
    accentedHitsSuccessful = fightStats.get('accentedHitsSuccessful')
    takedownTotal = fightStats.get('takedownTotal')
    takedownSuccessful = fightStats.get('takedownSuccessful')
    accentedHitsPositionDistanceTotal = fightStats.get('accentedHitsPositionDistanceTotal')
    accentedHitsPositionDistanceSuccessful = fightStats.get('accentedHitsPositionDistanceSuccessful')
    accentedHitsPositionClinchTotal = fightStats.get('accentedHitsPositionClinchTotal')
    accentedHitsPositionClinchSuccessful = fightStats.get('accentedHitsPositionClinchSuccessful')
    accentedHitsPositionParterTotal = fightStats.get('accentedHitsPositionParterTotal')
    accentedHitsPositionParterSuccessful = fightStats.get('accentedHitsPositionParterSuccessful')

    try: hitsSuccessful_percent = hitsSuccessful/hitsTotal
    except ZeroDivisionError: hitsSuccessful_percent = np.nan

    try: accentedHitsSuccessful_percent = accentedHitsSuccessful/hitsTotal
    except ZeroDivisionError: accentedHitsSuccessful_percent = np.nan

    try: accentedHits_percent = accentedHitsTotal/hitsTotal
    except ZeroDivisionError: accentedHits_percent = np.nan

    try: takedownSuccessful_percent = takedownSuccessful/takedownTotal
    except ZeroDivisionError: takedownSuccessful_percent = np.nan

    try: accentedHitsPositionDistanceSuccessful_percent = accentedHitsPositionDistanceTotal/accentedHitsPositionDistanceSuccessful
    except ZeroDivisionError: accentedHitsPositionDistanceSuccessful_percent = np.nan

    try: accentedHitsPositionClinchSuccessful_percent = accentedHitsPositionClinchTotal/accentedHitsPositionClinchSuccessful
    except ZeroDivisionError: accentedHitsPositionClinchSuccessful_percent = np.nan

    try: accentedHitsPositionParterSuccessful_percent = accentedHitsPositionParterTotal/accentedHitsPositionParterSuccessful
    except ZeroDivisionError: accentedHitsPositionParterSuccessful_percent = np.nan

    try: takedowns_to_hits = takedownSuccessful / hitsSuccessful
    except ZeroDivisionError: takedowns_to_hits = np.nan

    try: HitsPositionDistance_to_hits = accentedHitsPositionDistanceSuccessful / hitsSuccessful
    except ZeroDivisionError: HitsPositionDistance_to_hits = np.nan

    try: HitsPositionClinch_to_hits = accentedHitsPositionClinchSuccessful / hitsSuccessful
    except ZeroDivisionError: HitsPositionClinch_to_hits = np.nan

    try: HitsPositionParter_to_hits = accentedHitsPositionParterSuccessful / hitsSuccessful
    except ZeroDivisionError: HitsPositionParter_to_hits = np.nan
    
    hitsPM = (60 * hitsTotal) / duration 
    accentedHitsPM = (60 * accentedHitsTotal) / duration 
    takedownsPM = (60 * takedownTotal) / duration 
    accentedHitsDistancePM = (60 * accentedHitsPositionDistanceTotal) / duration 
    accentedHitsClinchPM = (60 * accentedHitsPositionClinchTotal) / duration 
    accentedHitsParterPM = (60 * accentedHitsPositionParterTotal) / duration 
        
    hitsSuccessfulPM = (60 * hitsSuccessful) / duration 
    accentedHitsSuccessfulPM = (60 * accentedHitsSuccessful) / duration 
    takedownsSuccessfulPM = (60 * takedownSuccessful) / duration 
    accentedHitsDistanceSuccessfulPM = (60 * accentedHitsPositionDistanceSuccessful) / duration 
    accentedHitsClinchSuccessfulPM = (60 * accentedHitsPositionClinchSuccessful) / duration 
    accentedHitsParterSuccessfulPM = (60 * accentedHitsPositionParterSuccessful) / duration 
        
    return  hitsPM, accentedHitsPM, takedownsPM, \
            accentedHitsDistancePM, accentedHitsClinchPM, accentedHitsParterPM, \
            hitsSuccessfulPM, accentedHitsSuccessfulPM, takedownsSuccessfulPM, \
            accentedHitsDistanceSuccessfulPM, accentedHitsClinchSuccessfulPM, accentedHitsParterSuccessfulPM, \
            hitsSuccessful_percent, accentedHitsSuccessful_percent, accentedHits_percent, \
            takedownSuccessful_percent, accentedHitsPositionDistanceSuccessful_percent, \
            accentedHitsPositionClinchSuccessful_percent, accentedHitsPositionParterSuccessful_percent, \
            takedowns_to_hits, HitsPositionDistance_to_hits, HitsPositionClinch_to_hits, \
            HitsPositionParter_to_hits 



In [6]:
fightStats_cols = ['hitsPM', 'accentedHitsPM', 'takedownsPM', 
            'accentedHitsDistancePM', 'accentedHitsClinchPM', 'accentedHitsParterPM', 
            
            'hitsSuccessfulPM', 'accentedHitsSuccessfulPM', 'takedownsSuccessfulPM', 
            'accentedHitsDistanceSuccessfulPM', 'accentedHitsClinchSuccessfulPM', 'accentedHitsParterSuccessfulPM',

            'hitsSuccessful_percent', 'accentedHitsSuccessful_percent', 'accentedHits_percent', 
            'takedownSuccessful_percent', 'accentedHitsPositionDistanceSuccessful_prcent',
            'accentedHitsPositionClinchSuccessful_percent',  'accentedHitsPositionParterSuccessful_percent', 
            'takedowns_to_hits', 'HitsPositionDistance_to_hits', 'HitsPositionClinch_to_hits', 
            'HitsPositionParter_to_hits',
            ]

fighter1_cols = [i+'_fighter1' for i in fightStats_cols]
fighter2_cols = [i+'_fighter2' for i in fightStats_cols]


In [7]:
df_stats = pd.DataFrame(index = df.index, columns=fighter1_cols + fighter2_cols + ['roundsCount'])

suppl_cols =  ['id', 'city', 'completed', 'country', 'duration',
               'eventDate.date','fighterId_1', 'fighterId_2', 'name', 'rounds',
               'timezone', 'weightCategory.name', 'winMethods',
               'winnerId', 'odd1', 'odd2', 'winner',
               ]

df_stats[suppl_cols] = df[suppl_cols]
df_stats

hitsPM_fighter1 accentedHitsPM_fighter1 takedownsPM_fighter1  \
0                NaN                     NaN                  NaN   
1                NaN                     NaN                  NaN   
2                NaN                     NaN                  NaN   
3                NaN                     NaN                  NaN   
4                NaN                     NaN                  NaN   
...              ...                     ...                  ...   
3729             NaN                     NaN                  NaN   
3730             NaN                     NaN                  NaN   
3731             NaN                     NaN                  NaN   
3732             NaN                     NaN                  NaN   
3733             NaN                     NaN                  NaN   

     accentedHitsDistancePM_fighter1 accentedHitsClinchPM_fighter1  \
0                                NaN                           NaN   
1                                NaN                           NaN   
2                                NaN                           NaN   
3                                NaN                           NaN   
4                                NaN                           NaN   
...                              ...                           ...   
3729                             NaN                           NaN   
3730                             NaN                           NaN   
3731                             NaN                           NaN   
3732                             NaN                           NaN   
3733                             NaN                           NaN   

     accentedHitsParterPM_fighter1 hitsSuccessfulPM_fighter1  \
0                              NaN                       NaN   
1                              NaN                       NaN   
2                              NaN                       NaN   
3                              NaN                       NaN   
4                              NaN                       NaN   
...                            ...                       ...   
3729                           NaN                       NaN   
3730                           NaN                       NaN   
3731                           NaN                       NaN   
3732                           NaN                       NaN   
3733                           NaN                       NaN   

     accentedHitsSuccessfulPM_fighter1 takedownsSuccessfulPM_fighter1  \
0                                  NaN                            NaN   
1                                  NaN                            NaN   
2                                  NaN                            NaN   
3                                  NaN                            NaN   
4                                  NaN                            NaN   
...                                ...                            ...   
3729                               NaN                            NaN   
3730                               NaN                            NaN   
3731                               NaN                            NaN   
3732                               NaN                            NaN   
3733                               NaN                            NaN   

     accentedHitsDistanceSuccessfulPM_fighter1  \
0                                          NaN   
1                                          NaN   
2                                          NaN   
3                                          NaN   
4                                          NaN   
...                                        ...   
3729                                       NaN   
3730                                       NaN   
3731                                       NaN   
3732                                       NaN   
3733                                       NaN   

     accentedHitsClinchSuccessfulPM_fighter1  \
0                                       

In [8]:
for i in tqdm(df.index[:]):
    fighters = df.loc[i, 'fighters']
    fighterId_1, fighterId_2, duration = df.loc[i, ['fighterId_1', 'fighterId_2', 'duration']]

    # Check index of fighters in dict
    if fighters[0]['fighterId'] == fighterId_1: 
        fighterId_1_ind = 0
        fighterId_2_ind = 1
    else:
        fighterId_1_ind = 1
        fighterId_2_ind = 0
        
    fighter1_stats = parse_data_from_fight(fighters[fighterId_1_ind]['fightStats'], duration)
    fighter2_stats = parse_data_from_fight(fighters[fighterId_2_ind]['fightStats'], duration)
    
    df_stats.loc[i, fighter1_cols] = fighter1_stats
    df_stats.loc[i, fighter2_cols] = fighter2_stats
    
df_stats    

/tmp/ipykernel_25590/1517480102.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm(df.index[:]):


  0%|          | 0/3734 [00:00<?, ?it/s]

hitsPM_fighter1 accentedHitsPM_fighter1 takedownsPM_fighter1  \
0          12.786885               12.786885                  0.0   
1               26.4                    20.6                  2.2   
2               22.2                    10.8                  2.6   
3           9.450549                5.934066             1.318681   
4               39.2                    18.4                  0.4   
...              ...                     ...                  ...   
3729            28.4                    16.0                  0.0   
3730             NaN                     NaN                  NaN   
3731             NaN                     NaN                  NaN   
3732        8.745763                3.457627              0.20339   
3733       27.403846               24.230769                  0.0   

     accentedHitsDistancePM_fighter1 accentedHitsClinchPM_fighter1  \
0                           4.918033                           0.0   
1                               14.2                           3.4   
2                                7.6                           1.2   
3                            0.21978                      1.538462   
4                               12.0                           3.8   
...                              ...                           ...   
3729                             8.0                           7.2   
3730                             NaN                           NaN   
3731                             NaN                           NaN   
3732                        3.254237                           0.0   
3733                       23.653846                           0.0   

     accentedHitsParterPM_fighter1 hitsSuccessfulPM_fighter1  \
0                         7.868852                 12.786885   
1                              3.0                      16.4   
2                              2.0                      16.8   
3                         4.175824                  7.692308   
4                              2.6                      26.6   
...                            ...                       ...   
3729                           0.8                      22.2   
3730                           NaN                       NaN   
3731                           NaN                       NaN   
3732                       0.20339                  5.491525   
3733                      0.576923                  9.230769   

     accentedHitsSuccessfulPM_fighter1 takedownsSuccessfulPM_fighter1  \
0                            12.786885                            0.0   
1                                 11.0                            1.0   
2                                  6.0                            1.0   
3                             4.395604                       0.659341   
4                                  8.2                            0.2   
...                                ...                            ...   
3729                              10.4                            0.0   
3730                               NaN                            NaN   
3731                               NaN                            NaN   
3732                          1.220339                        0.20339   
3733                          6.057692                            0.0   

     accentedHitsDistanceSuccessfulPM_fighter1  \
0                                     4.918033   
1                                          5.6   
2                                          3.2   
3                                          0.0   
4                                          3.0   
...                                        ...   
3729                                       4.0   
3730                                       NaN   
3731                                       NaN   
3732                                  1.016949   
3733                                  5.480769   

     accentedHitsClinchSuccessfulPM_fighter1  \
0                                       

In [9]:
def get_figher_statistics_from_past(df_stats, fighterId, eventDate):
    '''
    Aggregate statistics from fighter from the past.
    Check both positions 1 and 2.
    '''
    first_pos_df  = df_stats[(df_stats['fighterId_1'] == fighterId) & (df_stats['eventDate.date'] < eventDate)]
    first_pos_df  = pd.DataFrame(first_pos_df[fighter1_cols].values)

    second_pos_df = df_stats[(df_stats['fighterId_2'] == fighterId) & (df_stats['eventDate.date'] < eventDate)]
    second_pos_df = pd.DataFrame(second_pos_df[fighter2_cols].values)

    joined_df = first_pos_df.append(second_pos_df).reset_index(drop=True)

    return joined_df.mean()

valid_date = '2019-01-01 00:00:00'
test_date = '2020-01-01 00:00:00'

dates = [valid_date, test_date]
train_df_stats = df_stats[df_stats['eventDate.date'] < valid_date]
valid_df_stats = df_stats[(df_stats['eventDate.date'] > valid_date) & (df_stats['eventDate.date'] < test_date)]
test_df_stats = df_stats[df_stats['eventDate.date'] > test_date]

train_df_stats = df_stats[df_stats['eventDate.date'] < valid_date]
valid_df_stats = df_stats[(df_stats['eventDate.date'] > valid_date) & (df_stats['eventDate.date'] < test_date)]
test_df_stats = df_stats[df_stats['eventDate.date'] > test_date]

train_df_stats.shape, valid_df_stats.shape, test_df_stats.shape

((2622, 64), (501, 64), (611, 64))

In [10]:
train_df = train_df_stats.copy()
train_df['fighter1_fightsAmount'] = 0 
train_df['fighter2_fightsAmount'] = 0 

for i in tqdm(train_df_stats.index):
    fighterId_1, fighterId_2 = train_df.loc[i, ['fighterId_1', 'fighterId_2']]
    
    fighter1_history_stats = get_figher_statistics_from_past(train_df, fighterId_1, valid_date)
    train_df.loc[i, fighter1_cols] = fighter1_history_stats.values
    
    fighter2_history_stats = get_figher_statistics_from_past(train_df, fighterId_2, valid_date)
    train_df.loc[i, fighter2_cols] = fighter2_history_stats.values
    
    fighter1_fightsAmount = (((train_df['fighterId_1'] == fighterId_1) & (train_df['eventDate.date'] < valid_date)) | 
                             ((train_df['fighterId_2'] == fighterId_1) & (train_df['eventDate.date'] < valid_date))).sum()
    
    fighter2_fightsAmount = (((train_df['fighterId_1'] == fighterId_2) & (train_df['eventDate.date'] < valid_date)) 
                             |((train_df['fighterId_2'] == fighterId_2) & (train_df['eventDate.date'] < valid_date))).sum()
    
    train_df.loc[i, ['fighter1_fightsAmount', 
                       'fighter2_fightsAmount']] = fighter1_fightsAmount, fighter2_fightsAmount
train_df

/tmp/ipykernel_25590/902209873.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm(train_df_stats.index):


  0%|          | 0/2622 [00:00<?, ?it/s]

/tmp/ipykernel_25590/446012529.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  joined_df = first_pos_df.append(second_pos_df).reset_index(drop=True)
/tmp/ipykernel_25590/446012529.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  joined_df = first_pos_df.append(second_pos_df).reset_index(drop=True)
/tmp/ipykernel_25590/446012529.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  joined_df = first_pos_df.append(second_pos_df).reset_index(drop=True)
/tmp/ipykernel_25590/446012529.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  joined_df = first_pos_df.append(second_pos_df).reset_index(drop=True)
/tmp/ipykernel_25590/4460125

hitsPM_fighter1 accentedHitsPM_fighter1 takedownsPM_fighter1  \
0          21.488098               15.715088                  0.0   
1          30.651827               26.498256             2.412708   
2               21.0                    14.4                  2.6   
3          38.730283               35.616333             1.529616   
4          79.297047                64.70611             0.807491   
...              ...                     ...                  ...   
2617       35.880336               26.886581             2.296592   
2618       30.019841               29.647524             0.331688   
2619       17.768552               16.726238             1.021014   
2620       16.454797               10.494333             0.192593   
2621       10.689344                9.539344                  0.0   

     accentedHitsDistancePM_fighter1 accentedHitsClinchPM_fighter1  \
0                           12.58256                      1.257895   
1                          20.891113                      3.532143   
2                               12.3                           1.0   
3                          27.775974                      5.178815   
4                          58.069817                      3.369763   
...                              ...                           ...   
2617                         18.5169                      3.339109   
2618                       28.607934                      0.833401   
2619                       12.184433                      1.096329   
2620                        9.569032                      0.369746   
2621                        9.039344                           0.5   

     accentedHitsParterPM_fighter1 hitsSuccessfulPM_fighter1  \
0                         1.874633                 14.205776   
1                            2.075                 17.088787   
2                              1.1                      12.1   
3                         2.661544                 24.179319   
4                         3.266529                 42.049615   
...                            ...                       ...   
2617                      5.030572                  20.17403   
2618                       0.20619                 11.907286   
2619                      3.445476                  9.752363   
2620                      0.555556                  9.469076   
2621                           0.0                  4.613115   

     accentedHitsSuccessfulPM_fighter1 takedownsSuccessfulPM_fighter1  \
0                             9.478651                            0.0   
1                            13.185216                            0.4   
2                                  6.1                            0.6   
3                               21.202                       0.525191   
4                            28.860789                       0.291386   
...                                ...                            ...   
2617                         12.854511                       1.086845   
2618                          11.57101                       0.163101   
2619                          8.727021                        0.59536   
2620                          4.351807                        0.02963   
2621                          3.513115                            0.0   

     accentedHitsDistanceSuccessfulPM_fighter1  \
0                                     6.711035   
1                                     8.678073   
2                                          4.2   
3                                    15.045355   
4                                    23.676044   
...                                        ...   
2617                                  7.127744   
2618                                 10.899866   
2619                                  5.180509   
2620                                  3.781437   
2621                                  3.163115   

     accentedHitsClinchSuccessfulPM_fighter1  \
0                                   0.95

In [11]:
valid_df = valid_df_stats.copy()
valid_df['fighter1_fightsAmount'] = 0 
valid_df['fighter2_fightsAmount'] = 0 

for i in tqdm(valid_df_stats.index):
    fighterId_1, fighterId_2 = valid_df.loc[i, ['fighterId_1', 'fighterId_2']]
    
    fighter1_history_stats = get_figher_statistics_from_past(df_stats, fighterId_1, test_date)
    valid_df.loc[i, fighter1_cols] = fighter1_history_stats.values
    
    fighter2_history_stats = get_figher_statistics_from_past(df_stats, fighterId_2, test_date)
    valid_df.loc[i, fighter2_cols] = fighter2_history_stats.values
    
    fighter1_fightsAmount = (((valid_df['fighterId_1'] == fighterId_1) & (valid_df['eventDate.date'] < test_date)) | 
                             ((valid_df['fighterId_2'] == fighterId_1) & (valid_df['eventDate.date'] < test_date))).sum()
    
    fighter2_fightsAmount = (((valid_df['fighterId_1'] == fighterId_2) & (valid_df['eventDate.date'] < test_date)) 
                             |((valid_df['fighterId_2'] == fighterId_2) & (valid_df['eventDate.date'] < test_date))).sum()
    
    valid_df.loc[i, ['fighter1_fightsAmount', 
                       'fighter2_fightsAmount']] = fighter1_fightsAmount, fighter2_fightsAmount
    
valid_df

/tmp/ipykernel_25590/2907215782.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm(valid_df_stats.index):


  0%|          | 0/501 [00:00<?, ?it/s]

/tmp/ipykernel_25590/446012529.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  joined_df = first_pos_df.append(second_pos_df).reset_index(drop=True)
/tmp/ipykernel_25590/446012529.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  joined_df = first_pos_df.append(second_pos_df).reset_index(drop=True)
/tmp/ipykernel_25590/446012529.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  joined_df = first_pos_df.append(second_pos_df).reset_index(drop=True)
/tmp/ipykernel_25590/446012529.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  joined_df = first_pos_df.append(second_pos_df).reset_index(drop=True)
/tmp/ipykernel_25590/4460125

hitsPM_fighter1 accentedHitsPM_fighter1 takedownsPM_fighter1  \
2622       34.649763               29.065403              0.34218   
2623       11.348693               10.943702                  0.0   
2624       39.985714               37.742857             2.142857   
2625       43.599667               36.983476             1.452707   
2626       12.176294               11.243597             0.405405   
...              ...                     ...                  ...   
3118       41.259703               35.481877             0.281206   
3119       18.160799               16.478481             0.194824   
3120       20.366341               16.744553                  0.0   
3121       21.149848               19.995853              0.30794   
3122       61.533069               43.111508             1.612302   

     accentedHitsDistancePM_fighter1 accentedHitsClinchPM_fighter1  \
2622                       26.123223                       2.94218   
2623                        8.410272                      0.150754   
2624                       31.957143                           0.4   
2625                       25.088651                      4.234773   
2626                        9.652888                      1.185303   
...                              ...                           ...   
3118                       27.707593                      1.163081   
3119                       15.449006                      0.930303   
3120                       13.252374                      2.018994   
3121                       16.452993                      2.601156   
3122                       33.323413                      9.254762   

     accentedHitsParterPM_fighter1 hitsSuccessfulPM_fighter1  \
2622                           0.0                 19.321801   
2623                      2.382676                  5.655183   
2624                      5.385714                 15.257143   
2625                      7.660052                 25.072964   
2626                      0.405405                  4.384384   
...                            ...                       ...   
3118                      6.611203                 19.459609   
3119                      0.099171                  7.333237   
3120                      1.473184                 12.674651   
3121                      0.941704                 14.022136   
3122                      0.533333                 34.998545   

     accentedHitsSuccessfulPM_fighter1 takedownsSuccessfulPM_fighter1  \
2622                         14.179621                        0.14218   
2623                          5.400945                            0.0   
2624                         13.014286                       2.142857   
2625                         19.312444                       0.778292   
2626                          3.582406                       0.135135   
...                                ...                            ...   
3118                         14.224207                            0.0   
3119                          5.745309                       0.075758   
3120                          9.370461                            0.0   
3121                         12.868141                        0.30794   
3122                         18.414418                       0.345238   

     accentedHitsDistanceSuccessfulPM_fighter1  \
2622                                 11.679621   
2623                                  3.503109   
2624                                 12.214286   
2625                                 10.946674   
2626                                  2.780427   
...                                        ...   
3118                                  8.047232   
3119                                  5.139211   
3120                                  6.178282   
3121                                   9.94116   
3122                                 12.985847   

     accentedHitsClinchSuccessfulPM_fighter1  \
2622                                    

In [12]:
test_df = test_df_stats.copy()
test_df['fighter1_fightsAmount'] = 0 
test_df['fighter2_fightsAmount'] = 0 

for i in tqdm(test_df_stats.index):
    fighterId_1, fighterId_2 = test_df.loc[i, ['fighterId_1', 'fighterId_2']]
    
    fighter1_history_stats = get_figher_statistics_from_past(df_stats, fighterId_1, test_date)
    test_df.loc[i, fighter1_cols] = fighter1_history_stats.values
    
    fighter2_history_stats = get_figher_statistics_from_past(df_stats, fighterId_2, test_date)
    test_df.loc[i, fighter2_cols] = fighter2_history_stats.values
    
    fighter1_fightsAmount = (((test_df['fighterId_1'] == fighterId_1)) | 
                             ((test_df['fighterId_2'] == fighterId_1))).sum()
    
    fighter2_fightsAmount = (((test_df['fighterId_1'] == fighterId_2) ) 
                             |((test_df['fighterId_2'] == fighterId_2))).sum()
    
    test_df.loc[i, ['fighter1_fightsAmount', 
                       'fighter2_fightsAmount']] = fighter1_fightsAmount, fighter2_fightsAmount

/tmp/ipykernel_25590/3135421585.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm(test_df_stats.index):


  0%|          | 0/611 [00:00<?, ?it/s]

/tmp/ipykernel_25590/446012529.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  joined_df = first_pos_df.append(second_pos_df).reset_index(drop=True)
/tmp/ipykernel_25590/446012529.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  joined_df = first_pos_df.append(second_pos_df).reset_index(drop=True)
/tmp/ipykernel_25590/446012529.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  joined_df = first_pos_df.append(second_pos_df).reset_index(drop=True)
/tmp/ipykernel_25590/446012529.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  joined_df = first_pos_df.append(second_pos_df).reset_index(drop=True)
/tmp/ipykernel_25590/4460125

In [13]:
train_df.to_pickle('data/train_df.pkl')
valid_df.to_pickle('data/valid_df.pkl')
test_df.to_pickle('data/test_df.pkl')

In [14]:
train_df = pd.read_pickle('data/train_df.pkl')
valid_df = pd.read_pickle('data/valid_df.pkl')
test_df = pd.read_pickle('data/test_df.pkl')

In [15]:
def combine_df(to_combine_df):
    df_combined = pd.DataFrame(to_combine_df[fighter1_cols].values - to_combine_df[fighter2_cols].values, 
                               index=to_combine_df.index)
    
    df_combined.columns = fightStats_cols
    # df_combined = df_combined.fillna(0)
    df_combined['eventDate.date'] = to_combine_df['eventDate.date']
    df_combined['winner'] = to_combine_df['winner']
    df_combined['odd_diff'] = to_combine_df['odd1'] - to_combine_df['odd2']
    df_combined['fighter1_fightsAmount'] = to_combine_df['fighter1_fightsAmount']
    df_combined['fighter2_fightsAmount'] = to_combine_df['fighter2_fightsAmount']

    df_combined['odd1'] = to_combine_df['odd1']
    df_combined['odd2'] = to_combine_df['odd2']
#     df_combined = df_combined[~df_combined['odd_diff'].isna()]
    # to_combine_df = to_combine_df[(to_combine_df['fighter1_fightsAmount'] > 4) & (to_combine_df['fighter2_fightsAmount'] > 4)]
    return df_combined

train_df_combined = combine_df(train_df)
train_df_combined.to_pickle('data/train_df_combined.pkl')
train_df_combined

hitsPM accentedHitsPM takedownsPM accentedHitsDistancePM  \
0     18.537278      12.764268         0.0                9.63174   
1     19.651827      17.498256    1.212708              12.891113   
2           2.6           -1.0         2.6                   -2.3   
3     27.993521      32.747397    1.238924              26.768559   
4     55.068475      46.317538    0.647491               44.83709   
...         ...            ...         ...                    ...   
2617  23.202198      14.991686    1.435973               9.507907   
2618  -1.999532       1.923308   -0.194238               7.927592   
2619  -11.93686     -10.979639   -0.124804              -6.143978   
2620    -6.2409      -7.195372    0.192593              -6.511219   
2621  -6.261012      -1.618302    -0.67451               2.814802   

     accentedHitsClinchPM accentedHitsParterPM hitsSuccessfulPM  \
0                1.257895             1.874633        14.205776   
1                2.732143                1.875        12.888787   
2                     0.6                  0.7              6.1   
3                3.805099              2.17374        16.129135   
4                1.649763            -0.169315         29.77533   
...                   ...                  ...              ...   
2617             1.919166             3.564612        13.467191   
2618            -1.661186            -4.343098        -7.265364   
2619              -1.9429            -2.892761        -7.579749   
2620            -0.328997            -0.355156          0.51006   
2621            -1.029187            -3.403917        -6.800728   

     accentedHitsSuccessfulPM takedownsSuccessfulPM  \
0                    9.478651                   0.0   
1                   10.785216                   0.0   
2                         3.1                   0.6   
3                   20.169109              0.385818   
4                   21.386503              0.251386   
...                       ...                   ...   
2617                 6.902649              0.898192   
2618                -3.689959             -0.192455   
2619                -6.886539              0.167594   
2620                 -0.94942               0.02963   
2621                -2.601927             -0.426667   

     accentedHitsDistanceSuccessfulPM accentedHitsClinchSuccessfulPM  \
0                            6.711035                       0.959649   
1                            7.078073                       2.732143   
2                                 2.0                            0.4   
3                           14.963723                       3.251075   
4                           18.837603                       1.727653   
...                               ...                            ...   
2617                         3.211232                       1.099856   
2618                         1.324937                      -1.505139   
2619                        -3.998876                      -1.279773   
2620                          -0.5226                      -0.526374   
2621                         0.489543                       -0.90989   

     accentedHitsParterSuccessfulPM hitsSuccessful_percent  \
0                          1.807967               0.672354   
1                             0.975               0.115341   
2                               0.7                0.23916   
3                          1.954311              -0.064325   
4                          0.821247               0.065643   
...                             ...                    ...   
2617                       2.591561              -0.011728   
2618                      -3.509757              -0.254464   
2619                       -1.60789              -0.018051   
2620                       0.099554              -0.005134   
2621                       -2.18158              -0.153798   

     accentedHitsSuccessful_percent accentedHits_percent  \
0                          0.492925         

In [16]:
valid_df_combined = combine_df(valid_df)
valid_df_combined.to_pickle('data/valid_df_combined.pkl')
valid_df_combined

hitsPM accentedHitsPM takedownsPM accentedHitsDistancePM  \
2622  -25.89794     -19.612038   -0.024196               1.899624   
2623   8.413548       8.093302   -1.621034               5.983601   
2624  -7.590965       1.043771    -0.55089               4.832289   
2625 -57.506089     -49.296309   -0.098447             -50.491969   
2626 -13.334967     -13.409557    0.355405              -3.249364   
...         ...            ...         ...                    ...   
3118  -2.840035      -7.628505   -0.259648             -13.296777   
3119 -11.663098     -10.051576   -0.288376              -6.577657   
3120 -33.480823     -35.842107   -0.252101             -34.181974   
3121   3.739086       4.619619     0.30794               3.676759   
3122  36.933069      25.311508    1.412302              19.523413   

     accentedHitsClinchPM accentedHitsParterPM hitsSuccessfulPM  \
2622             0.768617           -22.280279       -16.028251   
2623            -0.272975             2.382676         4.024911   
2624            -4.653893             0.865375       -10.658257   
2625            -2.983843             4.179503       -25.856508   
2626             0.375843           -10.536036       -13.812913   
...                   ...                  ...              ...   
3118            -0.097671             5.765943         4.640153   
3119            -2.030127            -1.443792         -8.69991   
3120            -1.258317            -0.401816       -11.349771   
3121             0.001156             0.941704         5.649491   
3122             9.254762            -3.466667        22.998545   

     accentedHitsSuccessfulPM takedownsSuccessfulPM  \
2622               -11.261337             -0.047324   
2623                 3.855419                   0.0   
2624                -3.237961              1.297714   
2625               -18.955591               0.50037   
2626               -13.756783              0.135135   
...                       ...                   ...   
3118                  0.32964             -0.057143   
3119                -7.243997             -0.103094   
3120               -13.393457                   0.0   
3121                 6.230024               0.30794   
3122                12.014418              0.145238   

     accentedHitsDistanceSuccessfulPM accentedHitsClinchSuccessfulPM  \
2622                         0.431535                       1.287368   
2623                          2.21182                      -0.103484   
2624                         1.852607                       -2.32251   
2625                       -19.852446                      -2.188483   
2626                        -2.909212                      -0.311535   
...                               ...                            ...   
3118                        -4.330109                      -0.003573   
3119                        -4.383578                      -1.699017   
3120                       -11.685426                      -1.156216   
3121                         4.703043                       0.854335   
3122                         8.985847                       4.995238   

     accentedHitsParterSuccessfulPM hitsSuccessful_percent  \
2622                     -12.980241              -0.065729   
2623                       1.747083               -0.05898   
2624                      -2.768057              -0.144841   
2625                       3.085339               0.102683   
2626                     -10.536036                -0.2878   
...                             ...                    ...   
3118                       4.663322               0.196731   
3119                      -1.161403              -0.156904   
3120                      -0.551816               0.163407   
3121                       0.672646               0.197531   
3122                      -1.966667               0.022719   

     accentedHitsSuccessful_percent accentedHits_percent  \
2622                      -0.055969         

In [17]:
test_df_combined = combine_df(test_df)
test_df_combined.to_pickle('data/test_df_combined.pkl')
test_df_combined

hitsPM accentedHitsPM takedownsPM accentedHitsDistancePM  \
3123  10.971455       5.637201    0.133333              -0.888398   
3124  17.120207       5.929299    -1.00819               2.879043   
3125        NaN            NaN         NaN                    NaN   
3126        NaN            NaN         NaN                    NaN   
3127   8.112056      10.747844    0.755347              10.318166   
...         ...            ...         ...                    ...   
3729 -19.237944     -14.875817    -1.25977             -13.054159   
3730  13.987078      12.957003    0.094026              11.243155   
3731   -5.90872      -10.93896    0.416667             -10.046318   
3732 -23.222727     -20.243182    0.752273             -15.613636   
3733      6.278       3.849014    0.717982              -0.171621   

     accentedHitsClinchPM accentedHitsParterPM hitsSuccessfulPM  \
3123             3.702762             2.822836        10.553039   
3124             1.351026              1.69923         13.59643   
3125                  NaN                  NaN              NaN   
3126                  NaN                  NaN              NaN   
3127            -0.652594             1.082272         0.242703   
...                   ...                  ...              ...   
3729            -2.052932             0.231275         -8.14936   
3730             0.513364             1.200484         6.650314   
3731            -3.559308             2.666667        -3.011748   
3732             0.520455                -5.15       -10.386364   
3733             0.505484             3.515152         1.878189   

     accentedHitsSuccessfulPM takedownsSuccessfulPM  \
3123                 7.185451              0.333333   
3124                 3.680227             -0.062332   
3125                      NaN                   NaN   
3126                      NaN                   NaN   
3127                 2.606983              0.469948   
...                       ...                   ...   
3729                -3.987233             -0.333333   
3730                 5.354037               0.15308   
3731                -7.125321              0.166667   
3732                -7.836364              0.290909   
3733                -0.389035              0.360903   

     accentedHitsDistanceSuccessfulPM accentedHitsClinchSuccessfulPM  \
3123                         2.416943                       2.323757   
3124                         2.505188                       0.737856   
3125                              NaN                            NaN   
3126                              NaN                            NaN   
3127                         2.416371                      -0.445272   
...                               ...                            ...   
3729                         -3.20489                      -0.853257   
3730                         3.302352                       1.435954   
3731                        -6.323432                      -2.885223   
3732                        -5.606818                       0.620455   
3733                        -3.503829                       0.311333   

     accentedHitsParterSuccessfulPM hitsSuccessful_percent  \
3123                       2.444751               0.099652   
3124                       0.437184               0.117482   
3125                            NaN                    NaN   
3126                            NaN                    NaN   
3127                       0.635885              -0.138005   
...                             ...                    ...   
3729                       0.070914              -0.034903   
3730                       0.615731              -0.006717   
3731                       2.083333               0.042738   
3732                          -2.85               0.082173   
3733                       2.803461               -0.11956   

     accentedHitsSuccessful_percent accentedHits_percent  \
3123                       0.074393         